In [1]:
from keras.models import *
from keras.layers import *
from keras.optimizers import RMSprop,SGD,Adadelta,Adam
from keras.preprocessing import image
from keras.callbacks import TensorBoard,History,EarlyStopping,CSVLogger
from keras.applications.vgg19 import VGG19,preprocess_input
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import h5py
import os

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [2]:
train_path = './train'
validation_path = './validation'
test_path = './test'
assert(os.path.exists(train_path))
assert(os.path.exists(validation_path))
assert(os.path.exists(test_path))


nb_train_samples = 20000
nb_val_samples = 2424
image_size = (224,224)
batch_size = 128
class_num = 10

In [3]:
def image_preprocess(img_path):

    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    return img

In [4]:
def test_model(model,test_path,csv='sample_submission.csv'):
    columns = ['img', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    df = pd.DataFrame(columns=columns)
    
    file_list = os.listdir(test_path)
    for i,file in enumerate(file_list):
        test_data = image_preprocess(test_path + '/' + file)
        y_pred = model.predict(test_data,batch_size=1,verbose=0)
        y_pred = np.clip(y_pred,0.001,0.999)
        y_pred = y_pred[0].tolist()
        df.loc[i] = [file] + y_pred
        
    df.to_csv(csv,index=None)

In [3]:
train_datagen = image.ImageDataGenerator(
                                         shear_range=0.15,
                                         rotation_range=15,
                                         height_shift_range=0.15,
                                         width_shift_range=0.15,
                                         channel_shift_range=10,
                                         preprocessing_function=preprocess_input,
                                         horizontal_flip=True)

validation_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory(
                                    train_path,
                                    target_size=image_size,
                                    batch_size=batch_size,
                                    class_mode='categorical',
                                    shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
                                    validation_path,
                                    target_size=image_size,
                                    batch_size=batch_size,                                    
                                    class_mode='categorical',
                                    shuffle=False)

Found 20000 images belonging to 10 classes.
Found 2424 images belonging to 10 classes.


In [5]:
vgg19_base_model = VGG19(weights='imagenet', include_top=False,input_shape=(224,224,3))

x = vgg19_base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
# x = GlobalAveragePooling2D()(x)
x = Dense(10, activation='softmax')(x)

vgg19_model = Model(inputs=vgg19_base_model.input, outputs=x)

In [6]:
for layer in vgg19_model.layers[:22]:
    layer.trainable = False

for i,layer in enumerate(vgg19_model.layers):
    print(i,layer.name,layer.trainable)

0 input_2 False
1 block1_conv1 False
2 block1_conv2 False
3 block1_pool False
4 block2_conv1 False
5 block2_conv2 False
6 block2_pool False
7 block3_conv1 False
8 block3_conv2 False
9 block3_conv3 False
10 block3_conv4 False
11 block3_pool False
12 block4_conv1 False
13 block4_conv2 False
14 block4_conv3 False
15 block4_conv4 False
16 block4_pool False
17 block5_conv1 False
18 block5_conv2 False
19 block5_conv3 False
20 block5_conv4 False
21 block5_pool False
22 flatten_2 True
23 dense_4 True
24 batch_normalization_3 True
25 dropout_3 True
26 dense_5 True
27 batch_normalization_4 True
28 dropout_4 True
29 dense_6 True


In [7]:
vgg19_model.compile(loss='categorical_crossentropy',
                    #optimizer=optimizers.SGD(lr=1e-3, momentum=0.9) #,decay=1e-6,nesterov=True),
                    #optimizer=optimizers.Adam(lr=1e-3),
                    optimizer=Adadelta(),
                    metrics=['accuracy'])

# fine-tune the model
vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=3)])

vgg19_model.save('vgg19_model_fc.h5')

Epoch 1/20
156/156 [==============================] - 636s 4s/step - loss: 1.4598 - acc: 0.5405 - val_loss: 0.8229 - val_acc: 0.7609
Epoch 2/20
156/156 [==============================] - 362s 2s/step - loss: 0.5769 - acc: 0.8100 - val_loss: 0.9650 - val_acc: 0.7456
Epoch 3/20
156/156 [==============================] - 350s 2s/step - loss: 0.3924 - acc: 0.8728 - val_loss: 0.9505 - val_acc: 0.7831
Epoch 4/20
156/156 [==============================] - 353s 2s/step - loss: 0.3283 - acc: 0.8937 - val_loss: 1.0995 - val_acc: 0.7530


In [9]:
vgg19_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), #,decay=1e-6,nesterov=True),
                    #optimizer=optimizers.Adam(lr=1e-3),
#                     optimizer=Adadelta(),
                    metrics=['accuracy'])

# fine-tune the model
vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=10,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[EarlyStopping(patience=5)])

Epoch 1/10
156/156 [==============================] - 357s 2s/step - loss: 0.2938 - acc: 0.9062 - val_loss: 0.8892 - val_acc: 0.7778
Epoch 2/10
156/156 [==============================] - 352s 2s/step - loss: 0.2842 - acc: 0.9080 - val_loss: 0.8978 - val_acc: 0.7652
Epoch 3/10
156/156 [==============================] - 351s 2s/step - loss: 0.2812 - acc: 0.9083 - val_loss: 0.9824 - val_acc: 0.7561
Epoch 4/10
156/156 [==============================] - 350s 2s/step - loss: 0.2875 - acc: 0.9059 - val_loss: 1.0097 - val_acc: 0.7578
Epoch 5/10
156/156 [==============================] - 349s 2s/step - loss: 0.2718 - acc: 0.9102 - val_loss: 1.0898 - val_acc: 0.7374
Epoch 6/10
156/156 [==============================] - 349s 2s/step - loss: 0.2688 - acc: 0.9134 - val_loss: 1.0991 - val_acc: 0.7361


In [10]:
vgg19_model.save('vgg19_model_fc.h5')

In [11]:
vgg19_model = load_model('vgg19_model_fc.h5')

for layer in vgg19_model.layers[:17]:
    layer.trainable = False
    
for layer in vgg19_model.layers[17:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9), #decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), #])
                                   EarlyStopping(patience=10)])

vgg19_model.save('vgg19_model_17.h5')

Epoch 1/50
156/156 [==============================] - 356s 2s/step - loss: 0.2189 - acc: 0.9318 - val_loss: 0.8412 - val_acc: 0.7769
Epoch 2/50
156/156 [==============================] - 350s 2s/step - loss: 0.1687 - acc: 0.9474 - val_loss: 0.7594 - val_acc: 0.7953
Epoch 3/50
156/156 [==============================] - 352s 2s/step - loss: 0.1403 - acc: 0.9559 - val_loss: 0.8393 - val_acc: 0.7805
Epoch 4/50
156/156 [==============================] - 353s 2s/step - loss: 0.1288 - acc: 0.9604 - val_loss: 0.7981 - val_acc: 0.7962
Epoch 5/50
156/156 [==============================] - 350s 2s/step - loss: 0.1235 - acc: 0.9634 - val_loss: 0.8200 - val_acc: 0.7805
Epoch 6/50
156/156 [==============================] - 351s 2s/step - loss: 0.1128 - acc: 0.9662 - val_loss: 0.7907 - val_acc: 0.7905
Epoch 7/50
156/156 [==============================] - 349s 2s/step - loss: 0.1012 - acc: 0.9706 - val_loss: 0.8603 - val_acc: 0.7814
Epoch 8/50
156/156 [==============================] - 350s 2s/step - 

In [12]:
vgg19_model = load_model('vgg19_model_17.h5')

for layer in vgg19_model.layers[:12]:
    layer.trainable = False
    
for layer in vgg19_model.layers[12:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), #])
                                   EarlyStopping(patience=10)])

vgg19_model.save('vgg19_model_12.h5')

Epoch 1/50
156/156 [==============================] - 363s 2s/step - loss: 0.0749 - acc: 0.9782 - val_loss: 0.6270 - val_acc: 0.8277
Epoch 2/50
156/156 [==============================] - 360s 2s/step - loss: 0.0685 - acc: 0.9805 - val_loss: 0.6549 - val_acc: 0.8184
Epoch 3/50
156/156 [==============================] - 359s 2s/step - loss: 0.0683 - acc: 0.9799 - val_loss: 0.6262 - val_acc: 0.8271
Epoch 4/50
156/156 [==============================] - 358s 2s/step - loss: 0.0568 - acc: 0.9858 - val_loss: 0.7306 - val_acc: 0.8023
Epoch 5/50
156/156 [==============================] - 359s 2s/step - loss: 0.0571 - acc: 0.9846 - val_loss: 0.7571 - val_acc: 0.8040
Epoch 6/50
156/156 [==============================] - 360s 2s/step - loss: 0.0548 - acc: 0.9844 - val_loss: 0.7404 - val_acc: 0.8031
Epoch 7/50
156/156 [==============================] - 360s 2s/step - loss: 0.0543 - acc: 0.9852 - val_loss: 0.7113 - val_acc: 0.8127
Epoch 8/50
156/156 [==============================] - 361s 2s/step - 

In [13]:
vgg19_model.compile(loss='categorical_crossentropy',
#                     optimizer=optimizers.SGD(lr=1e-4, momentum=0.9,decay=1e-6,nesterov=True),
                    optimizer=optimizers.Adam(lr=1e-4),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=50,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'), #])
                                   EarlyStopping(patience=10)])

Epoch 1/50
156/156 [==============================] - 367s 2s/step - loss: 0.1745 - acc: 0.9467 - val_loss: 0.6323 - val_acc: 0.7938
Epoch 2/50
156/156 [==============================] - 369s 2s/step - loss: 0.1094 - acc: 0.9675 - val_loss: 0.5727 - val_acc: 0.8715
Epoch 3/50
156/156 [==============================] - 362s 2s/step - loss: 0.0475 - acc: 0.9864 - val_loss: 0.4653 - val_acc: 0.8807
Epoch 4/50
156/156 [==============================] - 362s 2s/step - loss: 0.0397 - acc: 0.9887 - val_loss: 0.5990 - val_acc: 0.8807
Epoch 5/50
156/156 [==============================] - 364s 2s/step - loss: 0.0343 - acc: 0.9909 - val_loss: 0.4945 - val_acc: 0.8663
Epoch 6/50
156/156 [==============================] - 363s 2s/step - loss: 0.0286 - acc: 0.9925 - val_loss: 0.5221 - val_acc: 0.8611
Epoch 7/50
156/156 [==============================] - 362s 2s/step - loss: 0.0206 - acc: 0.9943 - val_loss: 0.5641 - val_acc: 0.8776
Epoch 8/50
156/156 [==============================] - 360s 2s/step - 

In [14]:
vgg19_model.save('vgg19_model_final.h5')

In [5]:
vgg19_model = load_model('vgg19_model_final.h5')

for layer in vgg19_model.layers[:12]:
    layer.trainable = False
    
for layer in vgg19_model.layers[12:]:
    layer.trainable = True
    
vgg19_model.compile(loss='categorical_crossentropy',
                    optimizer=optimizers.SGD(lr=1e-5, momentum=0.9),#decay=1e-6,nesterov=True),
#                     optimizer=optimizers.Adam(),
                    metrics=['accuracy'])  

vgg19_model.fit_generator(
                        train_generator,
                        steps_per_epoch=nb_train_samples // batch_size,
                        epochs=20,
                        validation_data=validation_generator,
                        validation_steps=nb_val_samples // batch_size,
                        callbacks=[TensorBoard(log_dir='./logs/vgg19_fine_tune'),
                                   CSVLogger('vgg19_fine_tune'),
                                   EarlyStopping(patience=3)])


Epoch 1/20
156/156 [==============================] - 373s 2s/step - loss: 0.0137 - acc: 0.9964 - val_loss: 0.5557 - val_acc: 0.8941
Epoch 2/20
156/156 [==============================] - 370s 2s/step - loss: 0.0127 - acc: 0.9967 - val_loss: 0.5700 - val_acc: 0.8855
Epoch 3/20
156/156 [==============================] - 370s 2s/step - loss: 0.0137 - acc: 0.9958 - val_loss: 0.5907 - val_acc: 0.8802
Epoch 4/20
156/156 [==============================] - 370s 2s/step - loss: 0.0109 - acc: 0.9967 - val_loss: 0.6367 - val_acc: 0.8776


In [6]:
vgg19_model.save('vgg19_model_final.h5')

In [5]:
vgg19_model = load_model('vgg19_model_final.h5')

test_model(vgg19_model,test_path,csv='vgg19_submission.csv')